# Topic Modeling

Christoph & Nadia

## Imports von hauptsächlich Gensim & SpaCy

In [1]:
from pprint import pprint
import gensim
from gensim import corpora, models, similarities
from gensim.models import LdaModel, LsiModel
from spacy_preprocessing.preprocess import Preprocess
from tabulate import tabulate

import numpy as np
import pickle
import pyLDAvis
import pyLDAvis.gensim

import os
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import widgets
from IPython.display import display, clear_output

from spacy_preprocessing.preprocess import Preprocess
import datetime

load_saved = False
num_topics = 4
epochs_to_train = 10

## Zusätzliche Preprocessing Funktionen

In [2]:
def is_blacklisted(word):
    return word in [
        'polizei', 'polizist', 'beamter', 'nr.', 'berlin', 'uhr', 'polizeimeldung',
        'nicht', 'jahr', 'jährige', 'jährig', 'jähriger', 'polizeiliche', 'polizeilich', '2015', '2016',
        '2014', '2017', '2018', 'polizeibeamter', '-', 'u.a.', 'z.b.', 'der', 'die', 'das', 'dem', 'den', 'diese', 'dieser',
        'diesen', 'diesem', 'um', 'für', 'eine', 'ein', 'einer', 'einen', 'einem', 'anderer', 'andere', 'anderen', 'anders'
    ]

def is_empty(word):
    return word.strip() == ''

def can_parse_date(word):
    try:
        datetime.datetime.strptime(word, '%d.%m.%Y')
        return True
    except ValueError:
        return False

def can_parse_num_int(word):
    try:
        int(word)
        return True
    except ValueError:
        return False 

In [3]:
def can_parse_num_float(word):
    try:
        float(word)
        return True
    except ValueError:
        return False

def preprocess_after(doc):
    return [
        word 
        for word in doc
        if not is_empty(word)
        and not is_blacklisted(word)
        and not can_parse_date(word)
        and not can_parse_num_int(word)
        and not can_parse_num_float(word)
    ]

## Daten aus pickle Datei laden und Preprocessing durchführen

In [4]:
items = pickle.load(open('./data/items.pkl', 'rb'))
texts = [report['text'] for report in items]
titles = [report['title'] for report in items]
ids = [report['_id'] for report in items]
urls = [report['url'] for report in items]
data = [report['text_pre_processed_v1'] for report in items]

In [5]:
clean_data = [preprocess_after(doc) for doc in data]

## Dictionary und Bag of Words aus jedem Dokument erstellen

In [6]:
if load_saved:
    id2word = corpora.Dictionary.load('./data/id2word')
    corpus = pickle.load(open('./data/corpus.pkl', 'rb'))
else:
    id2word = corpora.Dictionary(clean_data)
    corpus = [id2word.doc2bow(doc) for doc in clean_data]
    
    id2word.save('./data/id2word')
    pickle.dump(corpus, open('./data/corpus.pkl', 'wb'))

## LDA (Latent Dirichlet Allocation)- / LSI (Latent semantic indexing )-Modell

In [7]:
if load_saved:
    lda_model = LdaModel.load('./data/lda_model')
    lsi_model = LsiModel.load('./data/lsi_model')
else: 
    lda_model = LdaModel(
        corpus=corpus, 
        id2word=id2word,
        num_topics=num_topics,
        passes=epochs_to_train, 
        alpha='auto',
        eta='auto'
    )
    
    lsi_model = LsiModel(
        corpus=corpus, 
        id2word=id2word,
        num_topics=num_topics, 
        power_iters=epochs_to_train
    )
    
    
    lda_model.save('./data/lda_model')
    lsi_model.save('./data/lsi_model')

In [8]:
pprint(lda_model.print_topics())

[(0,
  '0.016*"mann" + 0.012*"ermittlung" + 0.011*"straße" + 0.010*"mutmaßlich" + '
  '0.010*"nehmen" + 0.009*"alarmieren" + 0.009*"nacht" + 0.008*"festnehmen" + '
  '0.007*"auto" + 0.007*"fahrzeug"'),
 (1,
  '0.030*"mann" + 0.016*"ermittlung" + 0.014*"flüchten" + 0.012*"unbekannte" + '
  '0.010*"verletzen" + 0.010*"gestern" + 0.010*"anschließend" + 0.009*"täter" '
  '+ 0.009*"frau" + 0.008*"richtung"'),
 (2,
  '0.013*"wohnung" + 0.012*"mann" + 0.011*"ermittlung" + '
  '0.009*"staatsanwaltschaft" + 0.007*"landeskriminalamt" + 0.007*"finden" + '
  '0.007*"frau" + 0.007*"tatverdächtige" + 0.006*"verdacht" + 0.006*"gestern"'),
 (3,
  '0.015*"ermittlung" + 0.015*"richtung" + 0.015*"straße" + 0.015*"schwer" + '
  '0.015*"krankenhaus" + 0.013*"verletzen" + 0.013*"fahren" + '
  '0.013*"feuerwehr" + 0.012*"stationär" + 0.010*"behandlung"')]


In [9]:
bow = id2word.doc2bow(['messer', 'verletzung'])
vec = lda_model[bow]
vec

[(0, 0.03443994), (1, 0.8997117), (2, 0.022853356), (3, 0.04299495)]

## Topics visualisieren

In [10]:
%%capture

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [11]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.062527  0.153058       1        1  32.771267
3     -0.194315 -0.131013       2        1  31.533442
0      0.042623  0.065065       3        1  20.115885
2      0.214220 -0.087110       4        1  15.579401, topic_info=     Category         Freq                       Term        Total  loglift  \
term                                                                          
304   Default  1565.000000             verkehrsunfall  1565.000000  30.0000   
70    Default  2498.000000                  feuerwehr  2498.000000  29.0000   
7     Default  3053.000000                   flüchten  3053.000000  28.0000   
209   Default  2350.000000                  stationär  2350.000000  27.0000   
9     Default  3798.000000                krankenhaus  3798.000000  26.0000   
222   Default  1792.000000                    wohnung  1792.000000  25.0000   
207   Default  3202.000000                     schwer  3202.000000  24.0000   
746   Default   745.000000         staatsanwaltschaft   745.000000  23.0000   
575   Default  1363.000000                   schlagen  1363.000000  22.0000   
159   Default  9161.000000                       mann  9161.000000  21.0000   
248   Default  1172.000000                   erfassen  1172.000000  20.0000   
229   Default  2256.000000                       auto  2256.000000  19.0000   
751   Default  1135.000000                angestellte  1135.000000  18.0000   
758   Default  1124.000000                     räuber  1124.000000  17.0000   
182   Default  2890.000000                   erleiden  2890.000000  16.0000   
303   Default  1053.000000  verkehrsermittlungsdienst  1053.000000  15.0000   
14    Default  4225.000000                   richtung  4225.000000  14.0000   
347   Default  1933.000000                 mutmaßlich  1933.000000  13.0000   
345   Default  1129.000000                     messer  1129.000000  12.0000   
367   Default  1005.000000                   fahrbahn  1005.000000  11.0000   
631   Default  2110.000000                   fahrzeug  2110.000000  10.0000   
69    Default   952.000000                      feuer   952.000000   9.0000   
532   Default  1277.000000                     finden  1277.000000   8.0000   
185   Default  1747.000000                 festnehmen  1747.000000   7.0000   
17    Default  3043.000000                 unbekannte  3043.000000   6.0000   
923   Default   953.000000                    löschen   953.000000   5.0000   
691   Default   918.000000                   bedrohen   918.000000   4.0000   
305   Default  4225.000000                  verletzen  4225.000000   3.0000   
251   Default  1519.000000                     fahrer  1519.000000   2.0000   
51    Default  2037.000000                      täter  2037.000000   1.0000   
...       ...          ...                        ...          ...      ...   
910    Topic4   131.729874                explodieren   136.726227   1.8220   
2445   Topic4   252.605881                   dringend   284.760803   1.7394   
413    Topic4   396.628265                erstmeldung   478.741241   1.6711   
1030   Topic4   450.345367                 haftbefehl   553.916809   1.6522   
5838   Topic4   188.168350                   november   208.158081   1.7583   
222    Topic4  1145.272095                    wohnung  1792.626587   1.4112   
3454   Topic4   179.771240                    oktober   200.636322   1.7494   
1182   Topic4   262.740173                    hinweis   321.704529   1.6568   
217    Topic4   536.305603                   verdacht   954.389404   1.2829   
1042   Topic4   452.600708                     führen   833.767517   1.2483   
532    Topic4   595.606445                     finden  1277.302002   1.0963   
412    Topic4   313.159302                  ermittler   499.281372   1.3928   
949    Topic4   255.388885                    einsatz   391.122437   1.4330   

## MatrixSimilarity erstellen

In [12]:
if load_saved:
    index = similarities.MatrixSimilarity.load('./data/index')
else:
    index = similarities.MatrixSimilarity(lda_model[corpus])
    index.save('./data/index')

## Die Suche

In [13]:
use_lsi = False

html_widget = widgets.HTML(
    value = ''
)

text_widget = widgets.Text()


def search(sender):  
    preprocess = Preprocess(sender.value)
    search_term_preprocessed = preprocess_after(preprocess.preprocess(sentence_split=False, with_pos=False))
    search_term_bow = id2word.doc2bow(search_term_preprocessed)
    
    if use_lsi:
        search_term_vec = lsi_model[search_term_bow]
    else:
        search_term_vec = lda_model[search_term_bow]
    
    
    sims = index[search_term_vec]
    similarity_list = list(zip(range(len(sims)), sims, texts, titles, ids, urls))
    similarity_list.sort(key=lambda x: x[1], reverse=True)
    
    results = [f'<h3>{result[3]}</h3><em>Id: {result[4]} | Similarity Score: {round(result[1] * 100, 2)} % | Link: <a target="_blank" href="{result[5]}">{result[5]}</a></em><br><p style="width: 80%;">{result[2]}</p>' for result in similarity_list[:10]]
    output = '<hr><br>'.join(results)
    
    html_widget.value = output

text_widget.on_submit(search)    
    
display(text_widget)
display(html_widget)    


Text(value='')

HTML(value='')